In [1]:
import tensorflow as tf

import keras

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%matplotlib inline

/home/anton/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.layers import Input, LSTM, RepeatVector, Dense, Embedding, Activation, Masking, SimpleRNN
from keras.layers.wrappers import TimeDistributed
from keras.models import Model, Sequential
from keras import backend as K

from keras.utils import to_categorical

In [3]:
import pickle

In [4]:
from sklearn.model_selection import train_test_split

### Constants&data

In [6]:
X = np.load("../data/Kal_all_CDR/RNN_data_filtred.npy").astype(int)
Y = np.load("../data/Kal_all_CDR//RNN_data_filtred_y.npy")
voc = pickle.load(open("../data/Kal_all_CDR/l_to_n", 'rb'))
#mask = np.zeros_like(X)

In [22]:
Y[200000:]

array([0, 0, 0, ..., 1, 1, 1])

In [23]:
train_X, test_X, train_y,  test_y = train_test_split(X[200000:], Y[200000:])

In [24]:
VOC_SIZE = len(voc)
emb_size = 30


#### Net

In [25]:
encoder_input = Input(shape=(None,))

#encoder_mask_layer = Masking(mask_value=3, input_shape=(t_s,))
#encoder_mask = encoder_mask_layer(encoder_input)

encoder_embedding_layer = Embedding(VOC_SIZE, emb_size, mask_zero=True)
encoder_embedding = encoder_embedding_layer(encoder_input)

encoder_recurent_layer_1 = LSTM(100,return_state=True, return_sequences=True )
encoder_recurent_1, encoder_h_1, encoder_c_1 = encoder_recurent_layer_1(encoder_embedding)

encoder_recurent_layer_2 = LSTM(100,return_state=True, return_sequences=True )
encoder_recurent_2, encoder_h_2, encoder_c_2 = encoder_recurent_layer_2(encoder_recurent_1)

encoder_dense_layer = Dense(300, activation='relu')
encoder_dense = encoder_dense_layer(encoder_h_2)

encoder_out_layer = Dense(2, activation='softmax')
encoder_out = encoder_out_layer(encoder_dense)

In [26]:
model = Model([encoder_input], encoder_out)

##### Learning

In [27]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [48]:
learning_progress = model.fit(train_X, to_categorical(train_y), batch_size=100,
                                    epochs=1,
                                    validation_split=0.05)

Train on 216735 samples, validate on 11408 samples
Epoch 1/1
  3300/216735 [..............................] - ETA: 12:32 - loss: 0.6250

KeyboardInterrupt: 

In [29]:
pred_0 = model.predict(test_X)

In [30]:
pred_0

array([[0.5600327 , 0.43996733],
       [0.54869723, 0.45130283],
       [0.54688525, 0.4531148 ],
       ...,
       [0.53268397, 0.46731603],
       [0.56994945, 0.43005058],
       [0.00261115, 0.9973889 ]], dtype=float32)

In [39]:
test_y[:20]

array([1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0])

In [41]:
pred_0[:,1][:20]

array([0.43996733, 0.45130283, 0.4531148 , 0.73042065, 0.45377198,
       0.8805033 , 0.45311323, 0.46771047, 0.44072407, 0.70271355,
       0.8723782 , 0.45031574, 0.45065013, 0.42854086, 0.43475154,
       0.91206414, 0.45674893, 0.44465545, 0.45213446, 0.4692577 ],
      dtype=float32)

In [37]:
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [43]:
accuracy_score(test_y, pred_0[:,1] > 0.5) 

0.6133100147275405

In [46]:
mask = test_y == 0

In [45]:
accuracy_score(test_y[mask], pred_0[mask][:,1] > 0.5) 

0.27944293813162435

In [47]:
accuracy_score(test_y[mask], pred_0[mask][:,1] > 0.5) 

0.9950955521731777